In [1]:
import os
import numpy as np
import dask.dataframe as dd
import pandas as pd
import concurrent.futures
import warnings
import timeit
import time
warnings.filterwarnings("ignore")
import re
from geopy import distance

In [2]:
df = pd.read_csv('ZP_RM_61k_with_address_base.csv')

In [3]:
dff = pd.read_csv('ofcom_mobile_provider_2021-06.txt')

In [4]:
dff

,postcode,Provider,voice_indoor,voice_outdoor,voice_indoor_no4g,voice_outdoor_no4g,data_indoor,data_outdoor,enhanced_data_indoor,enhanced_data_outdoor,data_indoor_no4g,data_outdoor_no4g
0,B65 0BY,Vodafone,G,G,G,G,G,G,R,G,G,G
1,B65 0BY,Three,A,G,C,G,G,G,R,G,C,G
2,B65 0BZ,EE,A,G,A,G,G,G,R,G,A,G
3,B65 0BZ,O2,G,G,G,G,G,G,R,G,G,G
4,B65 0BZ,Vodafone,G,G,G,G,G,G,R,G,G,G
...,...,...,...,...,...,...,...,...,...,...,...,...
6723087,ZE3 9JY,Three,C,A,C,R,C,C,R,C,C,C
6723088,ZE3 9JZ,EE,C,G,C,G,A,G,R,G,C,G
6723089,ZE3 9JZ,O2,G,G,G,G,G,G,R,G,G,G
6723090,ZE3 9JZ,Vodafone,G,G,G,G,G,G,R,G,G,G


In [5]:
my_map = {}

In [6]:
for x in range(len(dff.index)):
    if x%200000 == 0:
        print(x)
    cur = dff.iloc[x]['postcode']
    if cur in my_map.keys():
        my_map[cur].append(dff.iloc[x])
    else:
        my_map[cur] = [dff.iloc[x]]

0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2200000
2400000
2600000
2800000
3000000
3200000
3400000
3600000
3800000
4000000
4200000
4400000
4600000
4800000
5000000
5200000
5400000
5600000
5800000
6000000
6200000
6400000
6600000


In [8]:
cols = list(dff.columns)
cols.remove('postcode')
cols.remove('Provider')

In [9]:
cols

['voice_indoor',
 'voice_outdoor',
 'voice_indoor_no4g',
 'voice_outdoor_no4g',
 'data_indoor',
 'data_outdoor',
 'enhanced_data_indoor',
 'enhanced_data_outdoor',
 'data_indoor_no4g',
 'data_outdoor_no4g']

In [11]:
for x in cols:
    df['Vodafone_'+x] = '-'
    df['Three_'+x] = '-'
    df['EE_'+x] = '-'
    df['O2_'+x] = '-'

In [10]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [12]:
for x in range(len(df.index)):
    cur_postcode = df.iloc[x]['POSTCODE_LOCATOR']
    #res = dff[dff['postcode'] == cur_postcode]
    if cur_postcode not in my_map.keys():
        continue
    res = my_map[cur_postcode]
    for i in range(len(res)):
        cur_prov = res[i]['Provider']
        for u in cols:
            df.loc[x,cur_prov+'_'+u] = res[i][u]
    
    if x%1000 == 0:
        print(x)
        

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000


In [13]:
df

,Date,Property type,Price,Source,Address,KeyFeatures,latitude,longitude,Bedrooms,Property Url,...,Three_voice_indoor,Three_voice_outdoor,Three_voice_indoor_no4g,Three_voice_outdoor_no4g,Three_data_indoor,Three_data_outdoor,Three_enhanced_data_indoor,Three_enhanced_data_outdoor,Three_data_indoor_no4g,Three_data_outdoor_no4g
0,25 Jun 2021,"Detached, Freehold","£995,000",Land Registry,"41 Runcie Close, St Albans, Hertfordshire AL4 9AX",Detached\nFour Bedrooms\nTwo Bathrooms\nStudy\...,51.770175,-0.313172,4,https://www.rightmove.co.uk/house-prices/detai...,...,G,G,A,G,G,G,R,G,A,G
1,10 Jun 2021,"Terraced, Freehold","£650,000",Land Registry,"37 Albany Road, Birmingham, West Midlands B17 9JX",VICTORIAN MID TERRACE\nFOUR BEDROOMS\nPERIOD F...,52.459072,-1.950948,4,https://www.rightmove.co.uk/house-prices/detai...,...,G,G,G,G,G,G,R,G,G,G
2,30 Jun 2021,"Detached, Freehold","£350,000",Land Registry,"19 Bibsworth Avenue, Birmingham, West Midlands...",Detached\nThree Bedrooms\nKitchen Diner\nTwo R...,52.432568,-1.859492,3,https://www.rightmove.co.uk/house-prices/detai...,...,G,G,G,G,G,G,R,G,G,G
3,18 Jun 2021,"Detached, Freehold","£382,500",Land Registry,"24 Sandy Croft, Birmingham, West Midlands B13 0EP",3 Bedrooms\nPorch\nEntrance Hall\nLounge\nKitc...,52.427717,-1.863037,3,https://www.rightmove.co.uk/house-prices/detai...,...,G,G,G,G,G,G,R,G,G,G
4,30 Jul 2021,"Semi-Detached, Freehold","£213,500",Land Registry,"15 Poppy Walk, Hatfield, Hertfordshire AL10 9FP",One Bedroomed House\nAllocated Parking\nChain ...,51.774284,-0.237790,1,https://www.rightmove.co.uk/house-prices/detai...,...,G,G,A,G,G,G,R,G,A,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61118,Aug 2021,NaN,"£225,000",NaN,53 Strathmore Avenue LU1 3NY,"Terraced house, Freehold, 3 Beds, 2 Baths, 2 R...",51.872773,-0.408768,3.0,https://www.zoopla.co.uk/property/53-strathmor...,...,G,G,G,G,G,G,R,G,G,G
61119,Aug 2021,NaN,"£60,000",NaN,13 Robinson Street DH8 5TR,"Terraced house, Freehold, 2 Beds, 1 Bath, 2 Re...",54.861234,-1.841458,3.0,https://www.zoopla.co.uk/property/13-robinson-...,...,G,G,A,G,G,G,R,G,A,G
61120,Aug 2021,NaN,"£125,000",NaN,10 Merrington Road TF2 8PY,"Semi-detached house, Freehold",52.725638,-2.419358,2.0,https://www.zoopla.co.uk/property/10-merringto...,...,C,G,C,G,A,G,R,G,C,G
61121,Aug 2021,NaN,"£318,000",NaN,"8 Mount View, Church Lane West GU11 3LN",NaN,51.243067,-0.767056,NaN,https://www.zoopla.co.uk/property/8-mount-view...,...,G,G,G,G,G,G,R,G,G,G


In [19]:
df.iloc[25468]

last_sold_date                                Jul 2021
Property type                                      NaN
last_sold_price                               £187,500
Source                                             NaN
Address                       10 Viaduct Drive WV6 0UX
                                        ...           
O2_data_indoor_no4g                                  G
Vodafone_data_outdoor_no4g                           G
Three_data_outdoor_no4g                              G
EE_data_outdoor_no4g                                 G
O2_data_outdoor_no4g                                 G
Name: 25468, Length: 143, dtype: object

In [14]:
dff = pd.read_csv('ofcom_broadband_2021-05.txt')

In [15]:
my_map_1 = {}

In [16]:
for x in range(len(dff.index)):
    if x%200000 == 0:
        print(x)
    cur = dff.iloc[x]['postcode']
    if cur in my_map_1.keys():
        my_map_1[cur].append(dff.iloc[x])
    else:
        my_map_1[cur] = [dff.iloc[x]]

0
200000
400000
600000
800000
1000000
1200000
1400000
1600000


In [17]:
cols = list(dff.columns)
cols.remove('postcode')

In [18]:
for x in cols:
    df[x] = '-'

In [20]:
for x in range(len(df.index)):
    cur_postcode = df.iloc[x]['POSTCODE_LOCATOR']
    #res = dff[dff['postcode'] == cur_postcode]
    if cur_postcode not in my_map_1.keys():
        continue
    res = my_map_1[cur_postcode]
    for u in cols:
        df.loc[x,u] = res[0][u]
    
    if x%1000 == 0:
        print(x)
        

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000


In [21]:
df

,Date,Property type,Price,Source,Address,KeyFeatures,latitude,longitude,Bedrooms,Property Url,...,Three_data_outdoor_no4g,max_adsl_predicted_down,max_adsl_predicted_up,max_sfbb_predicted_down,max_sfbb_predicted_up,max_ufbb_predicted_down,max_ufbb_predicted_up,adsl_availability,sfbb_availability,ufbb_availability
0,25 Jun 2021,"Detached, Freehold","£995,000",Land Registry,"41 Runcie Close, St Albans, Hertfordshire AL4 9AX",Detached\nFour Bedrooms\nTwo Bathrooms\nStudy\...,51.770175,-0.313172,4,https://www.rightmove.co.uk/house-prices/detai...,...,G,5,1,279,43,500,50,100,100,100
1,10 Jun 2021,"Terraced, Freehold","£650,000",Land Registry,"37 Albany Road, Birmingham, West Midlands B17 9JX",VICTORIAN MID TERRACE\nFOUR BEDROOMS\nPERIOD F...,52.459072,-1.950948,4,https://www.rightmove.co.uk/house-prices/detai...,...,G,16,1,200,20,1000,50,100,97,90
2,30 Jun 2021,"Detached, Freehold","£350,000",Land Registry,"19 Bibsworth Avenue, Birmingham, West Midlands...",Detached\nThree Bedrooms\nKitchen Diner\nTwo R...,52.432568,-1.859492,3,https://www.rightmove.co.uk/house-prices/detai...,...,G,16,1,67,16,1000,50,100,3,100
3,18 Jun 2021,"Detached, Freehold","£382,500",Land Registry,"24 Sandy Croft, Birmingham, West Midlands B13 0EP",3 Bedrooms\nPorch\nEntrance Hall\nLounge\nKitc...,52.427717,-1.863037,3,https://www.rightmove.co.uk/house-prices/detai...,...,G,13,1,38,8,1000,50,100,100,100
4,30 Jul 2021,"Semi-Detached, Freehold","£213,500",Land Registry,"15 Poppy Walk, Hatfield, Hertfordshire AL10 9FP",One Bedroomed House\nAllocated Parking\nChain ...,51.774284,-0.237790,1,https://www.rightmove.co.uk/house-prices/detai...,...,G,9,1,227,35,-1,-1,100,100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61118,Aug 2021,NaN,"£225,000",NaN,53 Strathmore Avenue LU1 3NY,"Terraced house, Freehold, 3 Beds, 2 Baths, 2 R...",51.872773,-0.408768,3.0,https://www.zoopla.co.uk/property/53-strathmor...,...,G,15,1,229,34,500,50,100,100,100
61119,Aug 2021,NaN,"£60,000",NaN,13 Robinson Street DH8 5TR,"Terraced house, Freehold, 2 Beds, 1 Bath, 2 Re...",54.861234,-1.841458,3.0,https://www.zoopla.co.uk/property/13-robinson-...,...,G,16,1,80,20,500,35,100,100,100
61120,Aug 2021,NaN,"£125,000",NaN,10 Merrington Road TF2 8PY,"Semi-detached house, Freehold",52.725638,-2.419358,2.0,https://www.zoopla.co.uk/property/10-merringto...,...,G,28,1,80,20,500,35,100,100,100
61121,Aug 2021,NaN,"£318,000",NaN,"8 Mount View, Church Lane West GU11 3LN",NaN,51.243067,-0.767056,NaN,https://www.zoopla.co.uk/property/8-mount-view...,...,G,15,1,80,20,500,35,100,100,92


In [22]:
df.to_csv('ZP_RM_61k_with_address_base_ofcom_added.csv')

In [20]:
list(df.columns)

['level_0',
 'index',
 'date',
 'address',
 'status',
 'beds',
 'type',
 'agent',
 'price',
 'link',
 'postcode',
 'new_home',
 'stc',
 'date_listed',
 'full_postcode',
 'folder_name',
 'full_address',
 'zoopla_link',
 'epc_values',
 'UPRN',
 'UDPRN',
 'CHANGE_TYPE',
 'STATE',
 'STATE_DATE',
 'CLASS',
 'PARENT_UPRN',
 'X_COORDINATE',
 'Y_COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'RPC',
 'LOCAL_CUSTODIAN_CODE',
 'COUNTRY',
 'LA_START_DATE',
 'LAST_UPDATE_DATE',
 'ENTRY_DATE',
 'RM_ORGANISATION_NAME',
 'LA_ORGANISATION',
 'DEPARTMENT_NAME',
 'LEGAL_NAME',
 'SUB_BUILDING_NAME',
 'BUILDING_NAME',
 'BUILDING_NUMBER',
 'SAO_START_NUMBER',
 'SAO_START_SUFFIX',
 'SAO_END_NUMBER',
 'SAO_END_SUFFIX',
 'SAO_TEXT',
 'ALT_LANGUAGE_SAO_TEXT',
 'PAO_START_NUMBER',
 'PAO_START_SUFFIX',
 'PAO_END_NUMBER',
 'PAO_END_SUFFIX',
 'PAO_TEXT',
 'ALT_LANGUAGE_PAO_TEXT',
 'USRN',
 'USRN_MATCH_INDICATOR',
 'AREA_NAME',
 'LEVEL',
 'OFFICIAL_FLAG',
 'OS_ADDRESS_TOID',
 'OS_ADDRESS_TOID_VERSION',
 'OS_ROADLINK_TOID'

In [18]:
df.to_csv('sample_data_2k_res_estimatemisses.csv')

In [66]:
df.to_excel('lilit_epc_school_transport_est_ofcom.xlsx')